In [1]:
import LotData.LotData as ld
import LotData.ExtractorsAndTables as ent
import utility.webscrapingUtil as wut
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine
import CW_Scraper
import time
import numpy as np
from sqlalchemy.orm import sessionmaker
from sqlalchemy.sql import text

In [2]:
engine = create_engine()
Session = sessionmaker(bind=engine)
session = Session()

"""
SECRET CODE THAT GENERATES MY DATAFRAME

"""

df.to_sql(record_key.replace("_record",""), con=engine, if_exists='append',index=False)

In [3]:
err_lids = []
for (cat_int,cat_name) in [(443,"wine"),(715,"diamonds"),(333,"watches"),(599,"gemstones")]:
    items = 0
    wttches_magov = CW_Scraper.MagazineOverview(cat_int)
    wttches_magov.set_active_nr_pages()
    for i,page_i in enumerate(wttches_magov):
        times = []
        for LID in page_i:
            t1 = time.time()
            meta_data = ent.MetadataExtractor(LID,wut.getTimeStamp(),cat_int,cat_name)

            l_data = ld.LotData(meta_data)
            for record_key in l_data.keys():
                if(record_key == "spec_record" or record_key == "auction_record"):
                    a = 2

                    """
                        result = session.execute(text("SELECT EXISTS (SELECT 1 FROM spec WHERE lid = :lid)"), {'lid': LID})
                        exists = result.scalar()  # This will be True if the LID exists, otherwise False
                        if(not exists):
                            try:
                                df = l_data[record_key]
                                df.to_sql(record_key.replace("_record",""), con=engine, if_exists='append',index=False)
                            except Exception as e:
                                err_lids += [LID]"""
                else:
                    try:
                        df = l_data[record_key]
                        df.to_sql(record_key.replace("_record",""), con=engine, if_exists='append',index=False)
                    except Exception as e:
                        err_lids += [(LID,e)]

            times += [time.time() - t1]
            time.sleep(np.random.uniform(0,0.01))
        print(f"Page number: {i} out of {len(wttches_magov)} of category {cat_name} \t Average time lot: {np.mean(times)} with variance {np.var(times)}")


Page number: 0 out of 211 of category wine 	 Average time lot: 0.8404465516408285 with variance 0.00704402982098726
Page number: 1 out of 211 of category wine 	 Average time lot: 0.7684807082017263 with variance 0.010557551678623059


KeyboardInterrupt: 

In [3]:
wttches_magov = CW_Scraper.MagazineOverview(443)
wttches_magov.set_active_nr_pages()
cat_int = 443
cat_name = "wine"

In [4]:
meta_data = ent.MetadataExtractor(LID,wut.getTimeStamp(),cat_int,cat_name)

In [11]:
lod = ld.LotData(meta_data)
ship_df = lod["shipping_record"]
ship_df.to_sql("shipping_record".replace("_record",""), con=engine, if_exists='append',index=False)

IntegrityError: (psycopg2.errors.UniqueViolation) duplicate key value violates unique constraint "shipping_lid_price_currency_code_key"
DETAIL:  Key (lid, price, currency_code)=(85238091, 120.0, EUR) already exists.

[SQL: INSERT INTO shipping (region_code, region_name, price, currency_code, estimated_delivery_from_days, estimated_delivery_to_days, destination_country_name, destination_country_short_code, combined_shipping_allowed, delivery_methods, extra_insurance, pr ... 13851 characters truncated ... _location_country_code__27)s, %(pickup_location_city__27)s, %(shipping_timestamp__27)s, %(lid__27)s)]
[parameters: {'extra_insurance__0': False, 'destination_country_name__0': 'Denmark', 'region_name__0': 'Switzerland', 'pickup_location_country_code__0': 'None', 'region_code__0': 'ch', 'shipping_timestamp__0': datetime.datetime(2024, 6, 22, 15, 55, 43, 942314, tzinfo=datetime.timezone.utc), 'combined_shipping_allowed__0': False, 'pickup_location_city__0': 'None', 'destination_country_sh

In [9]:
ship_df

,region_code,region_name,price,currency_code,estimated_delivery_from_days,estimated_delivery_to_days,destination_country_name,destination_country_short_code,combined_shipping_allowed,delivery_methods,extra_insurance,provider_id,is_pickup_preferable,is_pickup_only,pickup_location_country_code,pickup_location_city,shipping_timestamp,lid
0,ch,Switzerland,55.0,EUR,4,12,Denmark,dk,False,ship,False,1,False,False,None,None,2024-06-22 15:50:34.697869+00:00,85238091
1,cn,China,250.0,EUR,4,12,Denmark,dk,False,ship,False,1,False,False,None,None,2024-06-22 15:50:34.697869+00:00,85238091
2,cy,Cyprus,100.0,EUR,4,12,Denmark,dk,False,ship,False,1,False,False,None,None,2024-06-22 15:50:34.697869+00:00,85238091
3,fr,France,65.0,EUR,4,12,Denmark,dk,False,ship,False,1,False,False,None,None,2024-06-22 15:50:34.697869+00:00,85238091
4,gb,United Kingdom,120.0,EUR,4,12,Denmark,dk,False,ship,False,1,False,False,None,None,2024-06-22 15:50:34.697869+00:00,85238091
5,gg,Guernsey,120.0,EUR,4,12,Denmark,dk,False,ship,False,1,False,False,None,None,2024-06-22 15:50:34.697869+00:00,85238091
6,gr,Greece,100.0,EUR,4,12,Denmark,dk,False,ship,False,1,False,False,None,None,2024-06-22 15:50:34.697869+00:00,85238091
7,hk,Hong Kong,250.0,EUR,4,12,Denmark,dk,False,ship,False,1,False,False,None,None,2024-06-22 15:50:34.697869+00:00,85238091
8,ic,Canary Islands,110.0,EUR,4,12,Denmark,dk,False,ship,False,1,False,False,None,None,2024-06-22 15:50:34.697869+00:00,85238091
9,il,Israel,200.0,EUR,4,12,Denmark,dk,False,ship,False,1,False,False,None,None,2024-06-22 15:50:34.697869+00:00,85238091
